# Quick Start Guide

**Your First Deeplay Model**

Ready to dive in? This section will guide you through the process of creating your very first neural network model using Deeplay. We'll walk through all the essential steps, from creating a model to running a simple training loop.

### Step 1: Import Required Libraries

Start by importing Deeplay and other required libraries.

In [2]:
import deeplay as dl 
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


#### Understanding the Config Object

**Basic Syntax and Usage**

The `Config` object in `deeplay` serves as a flexible configuration tool for customizing neural network architectures. It allows you to set and adjust various parameters easily. Let's explore its basic syntax and how it can be utilized.

**Creating a Config Object**

You can set individual parameters by chaining methods. For example, to set the number of classes and the loss function:

In [6]:
from deeplay import Config

config = (
    Config()
    .num_classes(10)
    .loss(nn.CrossEntropyLoss)
    .loss.reduce("mean")
)

print(config)

Config(
#.num_classes = 10
#.loss = <class 'torch.nn.modules.loss.CrossEntropyLoss'>
loss.reduce = mean
)


In the above example, we make use of Deeplay's `Config` object to chain methods and seamlessly set multiple parameters for our model. Not only can you directly set classes or functions, such as `nn.CrossEntropyLoss`, you can also specify their arguments using attribute-like syntax. For instance, `.loss.reduce("mean")` sets the reduce argument of the CrossEntropyLoss to `"mean"`.

### Step 2: Create a Simple Model

Create a simple image classifier with Deeplay. We'll use a basic configuration for this example.

In [9]:
from deeplay import ImageClassifier, Config

config = Config().num_classes(10)
classifier = ImageClassifier.from_config(config)

print(classifier)

ImageClassifier(
  (backbone): ConvolutionalEncoder(
    (blocks): ModuleList(
      (0): Template(
        (layer): LazyConv2d(0, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Template(
        (layer): LazyConv2d(0, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Template(
        (layer): LazyConv2d(0, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (3): Template(
        (layer): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mo

### Step 3: Prepare Data

For demonstration purposes, we'll use the MNIST dataset. Here's how to load it and prepare data loaders.

In [10]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



### Step 4: Train the Model

Training a model in Deeplay is straightforward, especially if you're familiar with PyTorch Lightning. Here's a basic example to train your image classifier:

First, initialize your trainer:

In [13]:
from lightning import Trainer

trainer = Trainer(max_epochs=3, accelerator='cuda', devices=1)

# If you do not have a GPU, comment out the line above and uncomment the line below
# trainer = Trainer(max_epochs=3)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Next, fit the model to your data:

In [14]:
trainer.fit(classifier, train_loader)

c:\Users\GU\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\trainer\configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
Missing logger folder: c:\Users\GU\DeepTorch\notebooks\getting-started\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\GU\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\utilities\model_summary\model_summary.py:411: UserWarning: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.
  warning_cache.warn(

  | Name         | Type                          | Params
---------------------------------------------------------------
0 | backbone     | ConvolutionalEncoder          | 0     
1 | connector    | Flatten                       | 0     
2 | head         | CategoricalClassificationHead | 0     
3 | val_accuracy | Accuracy                      | 0     
----

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


Finally we test the model.

In [15]:
trainer.test(classifier, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\GU\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9815999865531921     │
│         test_loss         │    1.4801195859909058     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9815999865531921, 'test_loss': 1.4801195859909058}]

That's it! Deeplay's integration with PyTorch Lightning makes the training process smooth and efficient. The Trainer object takes care of most of the heavy lifting, allowing you to focus on refining your model architecture and hyperparameters.